# Installation
Please follow the instructions on README.mk file for installing the necessary packages to run this notebook

This walkthrough has few instructions. It's mainly just code to help the user to understand the pytorch geometric to hls4ml pipeline. If there's any confusion, please email me at yun79@purdue.edu

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch
import torch.nn as nn

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
import hls4ml

from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import GENConvBig
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size
import time
import pickle as pkl


2022-10-14 14:07:40.882053: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-14 14:07:40.882093: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


handler args: ('NodeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
handler args: ('NodeEncoderBatchNorm1d',)
handler args: ('EdgeEncoderBatchNorm1d',)
handler args: ('MeanPool',)
handler args: ('fc_out',)


### PyTorch Model

In [3]:
def number_of_parameters(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    print(f"N Model Parameters: {pp}")

In [113]:
"""
We intialize our custom pytorch geometric(pyg) model
"""
n_layers = 1
out_channels = 1
torch_model = GENConvBig(
    n_layers, 
    flow = "source_to_target",
    out_channels = out_channels,
    debugging = True
).eval() # eval mode for bathnorm

In [114]:
number_of_parameters(torch_model)

N Model Parameters: 26


In [115]:
torch_model.node_encoder_norm.weight = nn.Parameter(
    torch_model.node_encoder_norm.norm.weight
)

torch_model.node_encoder_norm.bias = nn.Parameter(
    torch_model.node_encoder_norm.norm.bias
)

torch_model.node_encoder_norm.running_mean = nn.Parameter(
    torch_model.node_encoder_norm.norm.running_mean
)

torch_model.node_encoder_norm.running_var = nn.Parameter(
    torch_model.node_encoder_norm.norm.running_var
)


In [116]:
torch_model.edge_encoder_norm.weight = nn.Parameter(
    torch_model.edge_encoder_norm.norm.weight
)

torch_model.edge_encoder_norm.bias = nn.Parameter(
    torch_model.edge_encoder_norm.norm.bias
)

torch_model.edge_encoder_norm.running_mean = nn.Parameter(
    torch_model.edge_encoder_norm.norm.running_mean
)

torch_model.edge_encoder_norm.running_var = nn.Parameter(
    torch_model.edge_encoder_norm.norm.running_var
)


In [117]:
Betas = []
for nodeblock_idx in range(n_layers):
    gnn = torch_model.gnns[nodeblock_idx]
    Betas.append(float(gnn.beta))

### HLS Model

hls4ml cannot infer the *order* in which these submodules are called within the pytorch model's "forward()" function. We have to manually define this information in the form of an ordered-dictionary.

In [118]:
"""
forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
"""
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
forward_dict["node_encoder_norm"] = "NodeEncoderBatchNorm1d"
forward_dict["edge_encoder_norm"] = "EdgeEncoderBatchNorm1d"
for nodeblock_idx in range(n_layers):
    forward_dict[f"O_{nodeblock_idx}"] = "NodeBlock"
forward_dict["pool"] = "MeanPool"  
forward_dict["fc_out"] = "fc_out"

hls4ml creates a hardware implementation of the GNN, which can only be represented using fixed-size arrays. This restriction also applies to the inputs and outputs of the GNN, so we must define the size of the graphs that this hardware GNN can take as input**, again in the form of a dictionary. 

**Graphs of a different size can be padded or truncated to the appropriate size using the "fix_graph_size" function. In this notebook, padding/truncation is  done in the "Data" cell. 

In [119]:
"""
we define additional parameters.
"""
common_dim = out_channels
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_attr": 3,
        "node_dim": common_dim,
        "edge_attr": 4,
    "edge_dim":common_dim
}

misc_config = {"Betas" : Betas}

Armed with our pytorch model and these two dictionaries**, we can create the HLS model. 

In [120]:
from hls4ml.model.optimizer import get_available_passes
"""
We initialize hls model from pyg model
"""
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<52,20>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)

# config["Optimizers"] = get_available_passes()

print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       misc_config = misc_config,
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid', #sigmoid
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<52,20>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
self.torch_model: GENConvBig(
  (node_encoder): Linear(in_features=3, out_features=1, bias=True)
  (node_encoder_norm): NodeEncoderBatchNorm1d(
    (norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (edge_encoder): Linear(in_features=4, out_features=1, bias=True)
  (edge_encoder_norm): EdgeEncoderBatchNorm1d(
    (norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (gnns): ModuleList(
    (0): GENConvSmall()
  )
  (O_0): ObjectModel(
    (layers): Sequential(
      (0): Linear(in_features=1, out_features=2, bias=True)
      (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=2, out_features=1, bias=True)
    )
  )
  (fc_out): Linear(in_features=1, out_features=1, bias=True)
)
mis

## hls_model.compile() builds the C-function for the model

In [121]:
hls_model.compile()
# hls_model.build()
"""
compile
build
implementation
"""

Writing HLS project
outputs_str: result_t layer12_out[N_LAYER_11]
layer: <hls4ml.model.hls_layers.Input object at 0x7fdaef459850>
layer: <hls4ml.model.hls_layers.Input object at 0x7fdaef459310>
layer: <hls4ml.model.hls_layers.Input object at 0x7fdaef459510>
layer: <hls4ml.model.hls_layers.NodeEncoder object at 0x7fdaef459250>
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
layer: <hls4ml.model.hls_layers.EdgeEncoder object at 0x7fdaef484410>
def_cpp: layer5_t layer5_out[N_LAYER_1_5*N_LAYER_2_5]
layer: <hls4ml.model.hls_layers.BatchNorm2D object at 0x7fdaf370c650>
def_cpp: layer6_t layer6_out[N_LAYER_1_4*N_LAYER_2_4]
layer: <hls4ml.model.hls_layers.BatchNorm2D object at 0x7fdaef459110>
def_cpp: layer7_t layer7_out[N_LAYER_1_5*N_LAYER_2_5]
layer: <hls4ml.model.hls_layers.EdgeAggregate object at 0x7fdaef4d5e90>
def_cpp: layer8_t layer8_out[N_NODE*LAYER8_OUT_DIM]
layer: <hls4ml.model.hls_layers.NodeBlock object at 0x7fdaef4a2750>
def_cpp: layer9_t layer9_out[N_LAYER_1_4*LAYER9_OUT_DI

'\ncompile\nbuild\nimplementation\n'

# Evaluation and prediction: hls_model.predict(input)

In [122]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
#         graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"

graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 2
writing test bench data for 1st graph


In [123]:
with open('test_data.pickle', 'rb') as f:
    graphs= pkl.load(f) 

MSEs = []
for data in graphs:
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
    hls_pred = hls_model.predict(data.hls_data)
    MSE = mean_squared_error(torch_pred, hls_pred)
    MSEs.append(MSE)
    
print(f"MSEs: \n {MSEs}")
print(f"Average MSEs: \n {np.mean(MSEs)}")

MSEs: 
 [1.1599538e-07, 1.1224763e-07, 7.792384e-09, 1.1599538e-07, 1.1599538e-07, 1.1599538e-07, 1.1245958e-07, 1.1016137e-07, 1.1245958e-07, 1.1245958e-07, 1.1583303e-07, 1.1599538e-07, 1.1599538e-07, 4.7070015e-10, 1.1599538e-07, 1.1245958e-07, 1.1599538e-07, 1.1245897e-07, 1.124595e-07, 1.1599538e-07]
Average MSEs: 
 1.0326101573809865e-07


In [124]:
import pandas as pd 

def compare_parameters(torch_parameter, hls_parameter):
    if not hls_parameter.endswith('.txt'): hls_parameter += '.txt'

    torch_parameter = torch_model.state_dict()[torch_parameter].T.numpy().flatten()
    
    hls_csv = os.path.join('test_GNN/firmware/weights/',hls_parameter)
    hls_parameter = np.genfromtxt(hls_csv, delimiter=',').flatten()

    torch_parameter = np.sort(torch_parameter)
    hls_parameter = np.sort(hls_parameter)

    try:
        torch.testing.assert_allclose(torch_parameter, hls_parameter)
    except AssertionError as e:
        print(e)
        return torch_parameter, hls_parameter
    return "We Good!"